In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import gc
import os
import sys
import time
warnings.filterwarnings('ignore')

print(os.getcwd())
code_path = '/root/ieee/pipeline/'
sys.path.append(code_path)


path = '/root/ieee/'

train_transaction = pd.read_csv(path + 'train_transaction.csv')
test_transaction = pd.read_csv(path + 'test_transaction.csv')

train_identity = pd.read_csv(path + 'train_identity.csv')
test_identity = pd.read_csv(path + 'test_identity.csv')
print("loading is over.")
train_transaction.sort_values('TransactionDT', inplace = True)
test_transaction.sort_values('TransactionDT', inplace = True)
train_transaction['nulls1'] = train_transaction.isna().sum(axis=1)
test_transaction['nulls1'] = test_transaction.isna().sum(axis=1)

card_feature = [col for col in train_transaction.columns if "card" in col] #category
addr_feature = [col for col in train_transaction.columns if "addr" in col] #category
dist_feature = [col for col in train_transaction.columns if "dist" in col] #numeric
mail_feature = [col for col in train_transaction.columns if "email" in col] #category

c_feature = [col for col in train_transaction.columns if "C" in col] #numeric
#C1-C14: counting, 
#    such as how many addresses are found to be associated with the payment card, etc. 
#The actual meaning is masked.
d_feature = [col for col in train_transaction.columns if "D" in col] #numeric 
#D1-D15: timedelta, such as days between previous transaction, etc.
d_feature.remove('TransactionID')
d_feature.remove('TransactionDT')
d_feature.remove('ProductCD')
c_feature.remove('ProductCD')
m_feature = [col for col in train_transaction.columns if "M" in col] #category
v_feature = [col for col in train_transaction.columns if "V" in col] #numeric

train_transaction = train_transaction.loc[train_transaction.card6 != 'debit or credit'].reset_index(drop = True)

C1_threshold = test_transaction.C1.mean() + test_transaction.C1.std() * 2.5
train_transaction = train_transaction[train_transaction.C1<C1_threshold].reset_index(drop = True)

# col = 'D1'
train_transaction['Transaction_day'] = np.floor((train_transaction['TransactionDT'] / (3600 * 24) - 1))
test_transaction['Transaction_day'] = np.floor((test_transaction['TransactionDT'] / (3600 * 24) - 1))
c_feat = [col for col in c_feature if col not in ['C13']]
# c_feat = [col for col in c_feature if col not in ['C13', 'C14']] #modify 2019.09.17 out of EDA

train_transaction['linear'] = train_transaction['Transaction_day']
test_transaction['linear'] = test_transaction['Transaction_day']
train_transaction['D4_new'] = train_transaction['D4']/test_transaction['linear']
test_transaction['D4_new'] = test_transaction['D4']/test_transaction['linear']
print("over.")

best_feature = pd.read_csv("/root/ieee/best_feat_v3.csv")
best_feature = list(best_feature.loc[best_feature.gain_score >= 0, 'feature'])

/root
loading is over.
over.


In [2]:
from feature import woe_encoder, timer, mail_func, addr_func, \
numeric_func, identity_func, \
product_func, match_func,card_func, \
all_category_encoding, Transaction_amt_encoding, \
C_feature, pca_missing, date_feature, D_feature


def feature_engineering(train_transaction, test_transaction):
    basic_feature = []
    data = pd.DataFrame([])
    tr_shape = train_transaction.shape[0]
    if mail_flag:
        with timer("1.mail related feature:"):
            temp = mail_func(train_transaction, test_transaction)
            data[temp.columns] = temp
            del temp;gc.collect()
    if num_flag:
        with timer("2.numeric type encoding feature:"):
            temp = numeric_func(train_transaction, test_transaction)
            data[temp.columns] = temp
            del temp;gc.collect()
    if iden_flag:
        with timer("3.identity flag count encoding:"):
            temp = identity_func(train_transaction, test_transaction, train_identity, test_identity)
            data[temp.columns] = temp
            del temp;gc.collect()
    if addr_flag:
        with timer("4.address type feature:"):
            temp = addr_func(train_transaction, test_transaction)
            data[temp.columns] = temp
            del temp;gc.collect()
    if prod_flag:
        with timer("5.product type feature:"):
            temp = product_func(train_transaction, test_transaction)
            data[temp.columns] = temp
            del temp;gc.collect()
    if match_flag:
        with timer("6.match type feature:"):
            temp = match_func(train_transaction, test_transaction)
            data[temp.columns] = temp
            del temp;gc.collect()      
    if card_flag:
        with timer("7.card type feature:"):
            temp = card_func(train_transaction, test_transaction)
            data[temp.columns] = temp
            del temp;gc.collect()      
    if category_flag:
        with timer("8.category type feature:"):
            temp = all_category_encoding(train_transaction, test_transaction)
            data[temp.columns] = temp
            del temp;gc.collect()
    if numeric_flag:
        with timer("9.numeric feature:"):
            temp = Transaction_amt_encoding(train_transaction, test_transaction)
            data[temp.columns] = temp
            del temp;gc.collect()
    if C_flag:
        with timer("10. C count feature:"):
            temp = C_feature(train_transaction, test_transaction)
            data[temp.columns] = temp
            del temp;gc.collect()
    if miss_flag:
        with timer("11. missing pca feature:"):
            temp = pca_missing(train_transaction, test_transaction)
            data[temp.columns] = temp
            del temp;gc.collect()
    if date_flag:
        with timer("12. time related feature:"):
            temp = date_feature(train_transaction, test_transaction)
            data[temp.columns] = temp
            del temp;gc.collect()
    if D_flag:
        with timer("13. D related feature:"):
            temp = D_feature(train_transaction, test_transaction)
            data[temp.columns] = temp
            del temp;gc.collect()
    if label_flag:
        with timer("14. Label encoding:"):
            temp = label_encoding(train_transaction, test_transaction)
            data[temp.columns] = temp
            del temp;gc.collect()
    if wise_flag:
        with timer("15. card wise anlaysis:"):
            temp = card_wise_exp(train_transaction, test_transaction)
            data[temp.columns] = temp
            del temp;gc.collect()
    if Dcount_flag:
        with timer("16. day_related_counting:"):
            temp = day_related_counting(train_transaction, test_transaction)
            data[temp.columns] = temp
            del temp;gc.collect()
        
    train_transaction[data.columns] = data[:tr_shape].reset_index(drop = True)
    test_transaction[data.columns] = data[tr_shape:].reset_index(drop = True)
    print("over.")
    return train_transaction, test_transaction, list(data.columns)


mail_flag = True
num_flag = True
iden_flag = True
addr_flag = True
prod_flag = True

match_flag = True
card_flag = True

category_flag = True
numeric_flag = True

C_flag = True
miss_flag = False
date_flag = True
D_flag = True
label_flag = False
wise_flag = False
Dcount_flag = False

train_transaction, test_transaction, basic_feature = feature_engineering(train_transaction, test_transaction)
features = c_feature + d_feature + dist_feature + v_feature + ['TransactionAmt','nulls1']
gc.collect()

del train_identity, test_identity;gc.collect()

1.mail related feature: - done in 5s
2.numeric type encoding feature: - done in 0s
3.identity flag count encoding: - done in 7s
4.address type feature: - done in 1s
5.product type feature: - done in 0s
6.match type feature: - done in 2s
7.card type feature: - done in 1s
8.category type feature: - done in 6s
9.numeric feature: - done in 10s
10. C count feature: - done in 1s
12. time related feature: - done in 2s
13. D related feature: - done in 1s
over.


0

In [3]:
%%time

def safe_div(a, b):
    try:
        return float(a) / float(b)
    except:
        return 0.0
def time_diff(tr, ts):
#     H_move = 12
    H_move = 0
    v_new = ['V{}'.format(i) for i in range(306, 312)]
    card_feature = [col for col in tr.columns if "card" in col] #category
    addr_feature = [col for col in tr.columns if "addr" in col] #category
    feat = ['TransactionID',"P_emaildomain", 'TransactionDT', 'TransactionAmt'] +\
            d_feature + card_feature + addr_feature + c_feature + v_new + dist_feature
    tr_ = tr[feat]
    ts_ = ts[feat]
    data_train = pd.DataFrame([])
    data_test = pd.DataFrame([])
    t1 = time.time()
    
    tr_['cid'] = \
    tr[c_feat + card_feature].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    ts_['cid'] = \
    ts[c_feat + card_feature].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

    tr_['uid'] = \
    tr[addr_feature + card_feature + ["P_emaildomain"]].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    ts_['uid'] = \
    ts[addr_feature + card_feature + ["P_emaildomain"]].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

#     tr_['uid'] = tr['card1'].astype(str) + "_" + tr['card2'].astype(str) +\
#                         tr['card3'].astype(str) + "_" + tr['card4'].astype(str) +\
#                         tr['card5'].astype(str) + "_" + tr['card6'].astype(str) +\
#                         tr['addr1'].astype(str) + "_" + tr['addr2'].astype(str) +\
#                         "_" + tr['P_emaildomain'].astype(str)
#     ts_['uid'] = ts['card1'].astype(str) + "_" + ts['card2'].astype(str) +\
#                         ts['card3'].astype(str) + "_" + ts['card4'].astype(str) +\
#                         ts['card5'].astype(str) + "_" + ts['card6'].astype(str) +\
#                         ts['addr1'].astype(str) + "_" + ts['addr2'].astype(str) +\
#                         "_" + ts['P_emaildomain'].astype(str)
    tr_["day"] = (tr["TransactionDT"] + 3600 * H_move) // (24 * 60 * 60)
    ts_["day"] = (ts["TransactionDT"] + 3600 * H_move) // (24 * 60 * 60)
    tr_['Hour'] = np.floor(tr['TransactionDT'] / 3600) % 24
    ts_['Hour'] = np.floor(ts['TransactionDT'] / 3600) % 24
    tr_['D1_delta'] = tr_['D1'] - tr_['day']
    ts_['D1_delta'] = ts_['D1'] - ts_['day']
    tr_['D2_delta'] = tr_['D2'] - tr_['day']
    ts_['D2_delta'] = ts_['D2'] - ts_['day']
    tr_['D10_delta'] = tr_['D10'] - tr_['day']
    ts_['D10_delta'] = ts_['D10'] - ts_['day']
    tr_['D15_delta'] = tr_['D15'] - tr_['day']
    ts_['D15_delta'] = ts_['D15'] - ts_['day']

    tr_['D1_delta_uid'] = tr_['D1_delta'].astype(str) + "_" + tr_['uid']
    tr_['D2_delta_uid'] = tr_['D2_delta'].astype(str) + "_" + tr_['uid']
    ts_['D1_delta_uid'] = ts_['D1_delta'].astype(str) + "_" + ts_['uid']
    ts_['D2_delta_uid'] = ts_['D2_delta'].astype(str) + "_" + ts_['uid']

    tr_['D10_delta_uid'] = tr_['D10_delta'].astype(str) + "_" + tr_['uid']
    tr_['D15_delta_uid'] = tr_['D15_delta'].astype(str) + "_" + tr_['uid']
    ts_['D10_delta_uid'] = ts_['D10_delta'].astype(str) + "_" + ts_['uid']
    ts_['D15_delta_uid'] = ts_['D15_delta'].astype(str) + "_" + ts_['uid']
    
    tr_['c4_d1_id'] = \
    tr_[['C4', 'D1_delta'] + card_feature].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    ts_['c4_d1_id'] = \
    ts_[['C4', 'D1_delta'] + card_feature].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    
#     tr_['cd_uid'] = \
#     tr_[['D1_delta_uid'] + c_feat].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
#     ts_['cd_uid'] = \
#     ts_[['D1_delta_uid'] + c_feat].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

    tr_['cd_uid'] = \
    tr_[['D1_delta_uid'] + ['C1', 'C4']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    ts_['cd_uid'] = \
    ts_[['D1_delta_uid'] + ['C1', 'C4']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

    t2 = time.time()
    print("diff:{}.s".format(t2 - t1))
    # TransactionAmt相关
    # 当天的交易次数
    data_train['trans_curday_Amt_cnt'] = tr_.groupby(['uid', 'day'])['TransactionAmt'].transform('count')
    data_test['trans_curday_Amt_cnt'] = ts_.groupby(['uid', 'day'])['TransactionAmt'].transform('count')
    # 当天的交易总额
    data_train['trans_curday_Amt_sum'] = tr_.groupby(['uid', 'day'])['TransactionAmt'].transform('sum')
    data_test['trans_curday_Amt_sum'] = ts_.groupby(['uid', 'day'])['TransactionAmt'].transform('sum')
    # 当天的交易最大金额
    data_train['trans_curday_Amt_max'] = tr_.groupby(['uid', 'day'])['TransactionAmt'].transform('max')
    data_test['trans_curday_Amt_max'] = ts_.groupby(['uid', 'day'])['TransactionAmt'].transform('max')
    # 当天的交易最小金额
    data_train['trans_curday_Amt_min'] = tr_.groupby(['uid', 'day'])['TransactionAmt'].transform('min')
    data_test['trans_curday_Amt_min'] = ts_.groupby(['uid', 'day'])['TransactionAmt'].transform('min')
    # 当天的交易平均金额
    data_train['trans_curday_Amt_mean'] = tr_.groupby(['uid', 'day'])['TransactionAmt'].transform('mean')
    data_test['trans_curday_Amt_mean'] = ts_.groupby(['uid', 'day'])['TransactionAmt'].transform('mean')
    # 当天同样金额的交易次数
    data_train['trans_curday_samAmt_cnt'] = tr_.groupby(['uid', 'day', 'TransactionAmt'])['TransactionAmt'].transform('count')
    data_test['trans_curday_samAmt_cnt'] = ts_.groupby(['uid', 'day', 'TransactionAmt'])['TransactionAmt'].transform('count')

    data_train['trans_curday_hour_max'] = tr_.groupby(['uid', 'day'])['Hour'].transform('max')
    data_test['trans_curday_hour_max'] = ts_.groupby(['uid', 'day'])['Hour'].transform('max')
    # 当天的交易最小Hour
    data_train['trans_curday_hour_min'] = tr_.groupby(['uid', 'day'])['Hour'].transform('min')
    data_test['trans_curday_hour_min'] = ts_.groupby(['uid', 'day'])['Hour'].transform('min')
    # 当天的交易平均Hour
    data_train['trans_curday_hour_mean'] = tr_.groupby(['uid', 'day'])['Hour'].transform('mean')
    data_test['trans_curday_hour_mean'] = ts_.groupby(['uid', 'day'])['Hour'].transform('mean')
    t3 = time.time()
    print("diff:{}.s".format(t3 - t2))
    
    
#     #当天同样金额的交易次数/全部
#     data_train['trans_curday_ratio'] = data_train['trans_curday_samAmt_cnt']/data_train['trans_curday_Amt_cnt']
#     data_test['trans_curday_ratio'] = data_test['trans_curday_samAmt_cnt']/data_test['trans_curday_Amt_cnt']
#     # 当天交易金额/最大金额
#     data_train['trans_Amt_ratio'] = tr_['TransactionAmt']/data_train['trans_curday_Amt_max']
#     data_test['trans_Amt_ratio'] = ts_['TransactionAmt']/data_test['trans_curday_Amt_max']
#     # 最小值/最大值
#     data_train['trans_Amt_max_min_ratio'] = data_train['trans_curday_Amt_min']/data_train['trans_curday_Amt_max']
#     data_test['trans_Amt_max_min_ratio'] = data_test['trans_curday_Amt_min']/data_test['trans_curday_Amt_max']
    
    data = pd.concat([data_train, data_test]).reset_index(drop = True)
    del data_train, data_test;gc.collect()
    # 距离上一笔以及下一笔交易的时间差特征(seconds)
    key_list = ['uid', 'D1_delta_uid', 'D2_delta_uid', 'c4_d1_id', 'cid']
#     values = ['TransactionDT', 'TransactionAmt'] + c_feature
    values = ['TransactionDT', 'TransactionAmt', 'C13', 'day']
#     values = ['TransactionDT', 'TransactionAmt', 'C13']
    df = pd.concat([tr_, ts_]).reset_index(drop = True)
    for key in key_list:
        for value in values:
            stat_temp = df[[key] + [value]].copy()
            for i in [-2, 2, -1, 1]:
                shift_value = stat_temp.groupby(key)[value].shift(i)
                cname = '_'.join([key, value]) + '_diff_time{}'.format(i)
                data[cname] = stat_temp[value] - shift_value
#                 cname = '_'.join([key, value]) + '_div_time{}'.format(i)
#                 data[cname] = stat_temp[value]/(0.01 + shift_value)

#     key_list = ['uid', 'D1_delta_uid', 'c4_d1_id', 'cid']    
    key_list = ['uid', 'D1_delta_uid', 'D2_delta_uid', 'c4_d1_id', 'cid'] #update from 19.09.16 20:52
#     stat_temp = df[key_list + d_feature+ ['TransactionAmt', 'C13'] + v_new].copy()
    for key in key_list:
        for col in ['D1', 'D2', 'D10', 'D15'] + ['TransactionAmt']:
            stat_temp = df[[key] + [col]].copy()
            for i in [-1, 1]:
                shift_value = stat_temp.groupby(key)[col].shift(i)
                cname = '_'.join([key, col]) +'_shift_time{}'.format(i)
                data[cname] = shift_value

    t4 = time.time()
    print("diff:{}.s".format(t4 - t3))
    
    return data

print(train_transaction.shape, test_transaction.shape)
tr_shape = train_transaction.shape[0]
new_feat = time_diff(train_transaction, test_transaction)
train_transaction[new_feat.columns] = new_feat[:tr_shape].reset_index(drop = True)
test_transaction[new_feat.columns] = new_feat[tr_shape:].reset_index(drop = True)
new_cols = list(new_feat.columns)
print(train_transaction.shape, test_transaction.shape)
del new_feat;gc.collect()
print("over.")



(587087, 542) (506691, 541)
diff:205.52618670463562.s
diff:30.166819095611572.s
diff:398.79793190956116.s
(587087, 681) (506691, 680)
over.
CPU times: user 10min 28s, sys: 26.2 s, total: 10min 54s
Wall time: 10min 50s


In [ ]:
%%time
def FE_OP_c1_c2(df, op = 'count', df_test = None):
    op = op.lower()
    c1, c2 = df.columns.tolist()
    c1c2 = 'FE_{}_({})_({})'.format(op.upper(), c1, c2)
    
    if df_test is not None:
        n_train = len(df)
        df = pd.concat([df, df_test], axis = 0).reset_index(drop = True)

    if op == 'count':
        s = df.groupby([c1, c2])[c1].transform(op)
    elif op in ['nunique', 'median', 'mean', 'min', 'max', 'std', 'sum', 'cumcount']:
        s = df.groupby(c1)[c2].transform(op)
    elif op == 'add':
        s = df[c1] + df[c2]
    elif op == 'diff':
        s = df[c1] - df[c2]
    elif op == 'mul':
        s = df[c1] * df[c2]
    elif op == 'div':
        s = df[c1] / df[c2]
    
    if df_test is not None:
        return c1c2, s[:n_train].reset_index(drop = True), s[n_train:].reset_index(drop = True)
    return c1c2, s

def decompose_name(col):
    c1, c2 = col.split(')_(')
    op, c1 = c1.split('_(')
    op = op[3:].lower()
    c2 = c2[:-1]
    return op, c1, c2

def name_to_fe(train, test, cols_add):
    for col in cols_add:
        print(col)
        op, c1, c2 = decompose_name(col)
        c1c2, train[col], test[col] =  FE_OP_c1_c2(train[[c1,c2]], op, test[[c1,c2]])
    return train, test

cols_add = [
'FE_MIN_(TransactionDT)_(C1)',
 'FE_MIN_(card2)_(C1)',
 'FE_MIN_(card1)_(C1)',
 'FE_MIN_(addr1)_(C1)',
 'FE_MIN_(card1)_(C13)',
 'FE_MIN_(card1)_(C14)',
 'FE_MIN_(card5)_(C1)',
 'FE_MAX_(TransactionDT)_(C1)',
 'FE_MAX_(card2)_(C1)',
 'FE_MAX_(card1)_(C1)',
 'FE_MAX_(addr1)_(C1)',
 'FE_MAX_(TransactionDT)_(C13)',
 'FE_MAX_(P_emaildomain)_(C1)',
 'FE_MAX_(card2)_(C13)',
 'FE_MAX_(card1)_(C13)',
 'FE_MAX_(TransactionDT)_(C14)',
 'FE_MAX_(TransactionDT)_(C12)',
 'FE_MAX_(addr1)_(C13)',
 'FE_MAX_(card2)_(C14)',
 'FE_MAX_(card1)_(C14)',
 'FE_MAX_(TransactionDT)_(TransactionAmt)',
 'FE_MAX_(card5)_(C1)']

print(train_transaction.shape, test_transaction.shape)
original_cols = list(train_transaction.columns)
train_transaction, test_transaction = name_to_fe(train_transaction, test_transaction, cols_add)
new_cols1 = [col for col in train_transaction.columns if col not in original_cols]
print(train_transaction.shape, test_transaction.shape)

(587087, 681) (506691, 680)
FE_MIN_(TransactionDT)_(C1)
FE_MIN_(card2)_(C1)
FE_MIN_(card1)_(C1)
FE_MIN_(addr1)_(C1)
FE_MIN_(card1)_(C13)
FE_MIN_(card1)_(C14)
FE_MIN_(card5)_(C1)
FE_MAX_(TransactionDT)_(C1)
FE_MAX_(card2)_(C1)
FE_MAX_(card1)_(C1)
FE_MAX_(addr1)_(C1)
FE_MAX_(TransactionDT)_(C13)
FE_MAX_(P_emaildomain)_(C1)
FE_MAX_(card2)_(C13)
FE_MAX_(card1)_(C13)
FE_MAX_(TransactionDT)_(C14)
FE_MAX_(TransactionDT)_(C12)
FE_MAX_(addr1)_(C13)
FE_MAX_(card2)_(C14)
FE_MAX_(card1)_(C14)
FE_MAX_(TransactionDT)_(TransactionAmt)
FE_MAX_(card5)_(C1)
(587087, 703) (506691, 702)
CPU times: user 41.7 s, sys: 1min 50s, total: 2min 32s
Wall time: 2min 32s


In [65]:
# %%time

def amt_accumulation_summary(tr, ts):
    v_list = ['V47', 'V244', 'V246', 'V257']
    key = card_feature + addr_feature + c_feature + d_feature + v_list
    df = pd.concat([tr[key + ['Transaction_day', 'TransactionDT','TransactionAmt','P_emaildomain']], 
                    ts[key + ['Transaction_day', 'TransactionDT','TransactionAmt','P_emaildomain']]]).reset_index(drop = True)
    H_move = 0
    df["Transaction_day"] = (df["TransactionDT"] + 3600 * H_move) // (24 * 60 * 60)
    col = 'D1'
    df[col + '_new'] = df[col] - df['Transaction_day']
    col = 'D2'
    df[col + '_new'] = df[col] - df['Transaction_day']
    data = pd.DataFrame([])
#     df['uid'] = df[card_feature + addr_feature].apply(
#                     lambda row: '_'.join(row.values.astype(str)), axis=1)

    df['uid'] = df[card_feature + addr_feature + ["P_emaildomain"]].apply(
                    lambda row: '_'.join(row.values.astype(str)), axis=1)
    df['d1_uid'] = df['uid'].astype(str) + "_" + df['D1_new'].astype(str)
    df['d2_uid'] = df['uid'].astype(str) + "_" + df['D2_new'].astype(str)
    
    df['c4_d1_id'] = \
    df[['C4', 'D1_new'] + card_feature].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    df['c14_d1_uid'] = \
    df[['C14', 'D1_new']+ card_feature].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    used = []
    for key in ['uid', 'd1_uid', 'c4_d1_id']:
        for col in ['TransactionAmt'] + ['D1']:
            df['{}_diff_1_{}'.format(col, key)] = \
            df[col] - df.groupby(key)[col].shift(1)
            df['{}_diff_-1_{}'.format(col, key)] = \
            df[col] - df.groupby(key)[col].shift(-1)
            used.append(('{}_diff_1_{}'.format(col, key), key))
            used.append(('{}_diff_-1_{}'.format(col, key), key))
    for func in ['mean', 'std', 'median']: #19.09.22
        for col in used:
            data["_".join([col[0], func])] = df.groupby(col[1])[col[0]].transform(func)

            
    new_used = []
    for key in ['uid', 'd1_uid', 'c4_d1_id']:
#         ['C1', 'C13', 'C14'] + ['D1_new']
        for col in ['C1', 'C14']:
            df['{}_encoding_{}'.format(col, key)] = df[col]
            new_used.append(('{}_encoding_{}'.format(col, key), key))

    for func in ['mean', 'std', 'median']: #19.09.22 
        for col in new_used:
            data["_".join([col[0], func])] = df.groupby(col[1])[col[0]].transform(func)
#     for key in ['c14_d1_uid']:
#         for col in ['TransactionAmt', 'TransactionDT']:
#             data['{}_diff_1_{}'.format(col, key)] = \
#             df[col] - df.groupby(key)[col].shift(1)
#             data['{}_diff_-1_{}'.format(col, key)] = \
#             df[col] - df.groupby(key)[col].shift(-1)
    return data

if new_cols2:
    train_transaction.drop(new_cols2, axis = 1, inplace = True)
    test_transaction.drop(new_cols2, axis = 1, inplace = True)
print(train_transaction.shape, test_transaction.shape)
tr_shape = train_transaction.shape[0]
new_feat = amt_accumulation_summary(train_transaction, test_transaction)
train_transaction[new_feat.columns] = new_feat[:tr_shape].reset_index(drop = True)
test_transaction[new_feat.columns] = new_feat[tr_shape:].reset_index(drop = True)
new_cols2 = list(new_feat.columns)
print(train_transaction.shape, test_transaction.shape)
del new_feat;gc.collect()
print("over.")

(587087, 720) (506691, 724)
(587087, 774) (506691, 778)
over.


In [ ]:
%%time
# a = train_transaction.groupby(card_feature + addr_feature)['isFraud'].expanding(5).sum().reset_index()

from joblib import Parallel, delayed


def expanding_helper(df, key, col, windows, func):
    g = pd.DataFrame([])
    name = "_".join([key, str(col), func, str(windows)])
    if func == 'sum':
        g[name] = df[col].apply(lambda x: x.expanding(windows).sum())
    if func == 'mean':
        g[name] = df[col].apply(lambda x: x.expanding(windows).mean())
    if (func == 'future_mean') & (col != 'isFraud'):
        cum_mean = df[col].apply(lambda x: x.expanding().mean())
        mean = df[col].transform('mean')
        g[name] = cum_mean - mean
    return g[name]

def expanding_feature(tr, ts):
    feat = ['TransactionID',"P_emaildomain", 'TransactionDT', 'TransactionAmt'] +\
            d_feature + card_feature + addr_feature + c_feature
    df = pd.concat([tr[feat + ['isFraud']], ts[feat]]).reset_index(drop = True)
    H_move = 0
    df["day"] = (df["TransactionDT"] + 3600 * H_move) // (24 * 60 * 60)
    df['D1_delta'] = df['D1'] - df['day']
    df['cid'] = \
    df[c_feat + card_feature + 
       addr_feature + ['D1_delta']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    df['uid'] = df[card_feature + addr_feature + ['P_emaildomain']].apply(
                    lambda row: '_'.join(row.values.astype(str)), axis=1)
    df['c4_d1_id'] = df[['C4', 'D1_delta'] + card_feature + addr_feature].apply(
                    lambda row: '_'.join(row.values.astype(str)), axis=1)
    data = pd.DataFrame([])
    cols_expanding = ['isFraud']
#     cols_expanding = ['TransactionAmt']
    for key in ['c4_d1_id']:
        if key == 'cid':
            windows = [50, 100, 150, 200]
        else:
            windows = [12, 24, 36]
        GroupBy = df.groupby([key])
        res = Parallel(n_jobs = 12, backend = 'multiprocessing') \
            (delayed(expanding_helper)(GroupBy, key, col, period, func) 
             for col in cols_expanding
             for period in windows
             for func in ['mean']
            )
        ans = pd.concat(res, axis = 1)
        data[ans.columns] = ans
    #counting
    #sum/mean ...
#     key_list = ['uid', 'c4_d1_id', 'cid']
#     for key in key_list:
#         for col in ['TransactionAmt']:
#             cumcount = df.groupby(key)[col].apply(lambda x: x.expanding().mean())
#             count_ = df.groupby(key)[col].transform('mean')
#             cname = "_".join([key, str(col)]) + "_future_mean"
#             data[cname] = count_ - cumcount
    return data

print(train_transaction.shape, test_transaction.shape)
tr_shape = train_transaction.shape[0]
new_feat = expanding_feature(train_transaction, test_transaction)
train_transaction[new_feat.columns] = new_feat[:tr_shape].reset_index(drop = True)
test_transaction[new_feat.columns] = new_feat[tr_shape:].reset_index(drop = True)
new_cols3 = list(new_feat.columns)
print(train_transaction.shape, test_transaction.shape)
del new_feat;gc.collect()
print("over.")

# if new_cols4:
#     train_transaction.drop(new_cols4, axis = 1, inplace = True)
#     test_transaction.drop(new_cols4, axis = 1, inplace = True)

In [100]:
# %%time

def amt_accumulation_standard(tr, ts):
    v_list = ['V47', 'V244', 'V246', 'V257']
    key = card_feature + addr_feature + c_feature + d_feature + v_list
    df = pd.concat([tr[key + ['Transaction_day', 'TransactionDT','TransactionAmt','P_emaildomain']], 
                    ts[key + ['Transaction_day', 'TransactionDT','TransactionAmt','P_emaildomain']]]).reset_index(drop = True)
    H_move = 0
    df["Transaction_day"] = (df["TransactionDT"] + 3600 * H_move) // (24 * 60 * 60)
    col = 'D1'
    df[col + '_new'] = df[col] - df['Transaction_day']
    col = 'D10'
    df[col + '_new'] = df[col] - df['Transaction_day']
    col = 'D15'
    df[col + '_new'] = df[col] - df['Transaction_day']
    
    data = pd.DataFrame([])
    df['uid'] = df[card_feature + addr_feature + ["P_emaildomain"]].apply(
                    lambda row: '_'.join(row.values.astype(str)), axis=1)
    df['d1_uid'] = df['uid'].astype(str) + "_" + df['D1_new'].astype(str)
    
    df['c4_d1_id'] = \
    df[['C4', 'D1_new'] + card_feature].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    df['d10_d1_uid'] = \
    df[['D10_new', 'D1_new']+ card_feature].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    df['d15_d1_uid'] = \
    df[['D15_new', 'D1_new']+ card_feature].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    df['most_accurate_id'] = \
    df[card_feature + addr_feature + ["P_emaildomain"] + c_feat + ['D1_new', 'D10_new']].apply(
                    lambda row: '_'.join(row.values.astype(str)), axis=1)
            
    new_used = []
    for key in ['uid', 'd1_uid', 'c4_d1_id'] + ['most_accurate_id']:
#          + ['C14', 'C1']
        for col in ['TransactionAmt']:
            df['{}_standard_{}'.format(col, key)] = df[col]
            new_used.append(('{}_standard_{}'.format(col, key), key))
            
    for func in ['mean', 'median']: #19.09.22 
        for col in new_used:
            data["_".join([col[0], func])] = df[col[0]] - df.groupby(col[1])[col[0]].transform(func)
#     for key in ['c14_d1_uid']:
#         for col in ['TransactionAmt', 'TransactionDT']:
#             data['{}_diff_1_{}'.format(col, key)] = \
#             df[col] - df.groupby(key)[col].shift(1)
#             data['{}_diff_-1_{}'.format(col, key)] = \
#             df[col] - df.groupby(key)[col].shift(-1)
    return data

if new_cols4:
    train_transaction.drop(new_cols4, axis = 1, inplace = True)
    test_transaction.drop(new_cols4, axis = 1, inplace = True)
print(train_transaction.shape, test_transaction.shape)
tr_shape = train_transaction.shape[0]
new_feat = amt_accumulation_standard(train_transaction, test_transaction)
train_transaction[new_feat.columns] = new_feat[:tr_shape].reset_index(drop = True)
test_transaction[new_feat.columns] = new_feat[tr_shape:].reset_index(drop = True)
new_cols4 = list(new_feat.columns)
print(train_transaction.shape, test_transaction.shape)
del new_feat;gc.collect()
print("over.")

(587087, 774) (506691, 778)
(587087, 782) (506691, 786)
over.


In [102]:
!cp ieee_moelling_with_rule_v5.ipynb /ieee/pipeline/

In [101]:

from model import kfold_lightgbm
used = [col for col in features + basic_feature + new_cols +\
        new_cols1 + new_cols2 + new_cols3 + new_cols4 if col not in d_feature]
print(len(used))
oof_train, oof_test, score_list = kfold_lightgbm(train_transaction, test_transaction, used)

727
train.shape = (587087, 782), test.shape = (506691, 786)
############################################################ fold = 1 / 5
####### cur time = 2019/09/23 09:48:11
Training until validation scores don't improve for 100 rounds.
[500]	valid_0's auc: 0.93203


KeyboardInterrupt: 

In [71]:

from model import kfold_lightgbm
used = [col for col in features + basic_feature + new_cols +\
        new_cols1 + new_cols2 + new_cols3 + new_cols4 if col not in d_feature]
print(len(used))
oof_train, oof_test, score_list = kfold_lightgbm(train_transaction, test_transaction, used)

725
train.shape = (587087, 780), test.shape = (506691, 784)
############################################################ fold = 1 / 5
####### cur time = 2019/09/22 21:59:34
Training until validation scores don't improve for 100 rounds.
[500]	valid_0's auc: 0.933458
[1000]	valid_0's auc: 0.9414
[1500]	valid_0's auc: 0.941969
Early stopping, best iteration is:
[1431]	valid_0's auc: 0.942064
period: [     0      1      2 ... 117415 117416 117417] , the score is 0.9420644452655844
############################################################ fold = 2 / 5
####### cur time = 2019/09/22 22:08:57
Training until validation scores don't improve for 100 rounds.
[500]	valid_0's auc: 0.959711
[1000]	valid_0's auc: 0.963384
Early stopping, best iteration is:
[1253]	valid_0's auc: 0.963631
period: [117418 117419 117420 ... 234833 234834 234835] , the score is 0.963631486634543
############################################################ fold = 3 / 5
####### cur time = 2019/09/22 22:17:00
Training unti

In [66]:

from model import kfold_lightgbm
used = [col for col in features + basic_feature + new_cols +\
        new_cols1 + new_cols2 + new_cols3 if col not in d_feature]
print(len(used))
oof_train, oof_test, score_list = kfold_lightgbm(train_transaction, test_transaction, used)

719
train.shape = (587087, 774), test.shape = (506691, 778)
############################################################ fold = 1 / 5
####### cur time = 2019/09/22 19:21:28
Training until validation scores don't improve for 100 rounds.
[500]	valid_0's auc: 0.933254
[1000]	valid_0's auc: 0.941416
Early stopping, best iteration is:
[1339]	valid_0's auc: 0.942209
period: [     0      1      2 ... 117415 117416 117417] , the score is 0.9422092898315451
############################################################ fold = 2 / 5
####### cur time = 2019/09/22 19:30:08
Training until validation scores don't improve for 100 rounds.
[500]	valid_0's auc: 0.959661
[1000]	valid_0's auc: 0.963228
Early stopping, best iteration is:
[1332]	valid_0's auc: 0.963522
period: [117418 117419 117420 ... 234833 234834 234835] , the score is 0.963521961364452
############################################################ fold = 3 / 5
####### cur time = 2019/09/22 19:38:21
Training until validation scores don't imp

In [ ]:

from model import kfold_lightgbm
used = [col for col in features + basic_feature + new_cols +\
        new_cols1 + new_cols2 + new_cols3 if col not in d_feature]
print(len(used))
oof_train, oof_test, score_list = kfold_lightgbm(train_transaction, test_transaction, used)

[500]	valid_0's auc: 0.932762
Early stopping, best iteration is:
[1239]	valid_0's auc: 0.968019
period: [352253 352254 352255 ... 469667 469668 469669] , the score is 0.9680192595089138
############################################################ fold = 5 / 5
####### cur time = 2019/09/22 10:06:07
Training until validation scores don't improve for 100 rounds.
[500]	valid_0's auc: 0.949395
[1000]	valid_0's auc: 0.952914
[1500]	valid_0's auc: 0.95359
Early stopping, best iteration is:
[1603]	valid_0's auc: 0.95365
period: [469670 469671 469672 ... 587084 587085 587086] , the score is 0.9536504764202409
score_list_mean = 0.957900
score full train = 0.957981


In [9]:
%%time
from post import online_rule_parallel
col = 'D1'
train_transaction[col + '_new'] = train_transaction[col] - train_transaction['Transaction_day']
test_transaction[col + '_new'] = test_transaction[col] - test_transaction['Transaction_day']
for col in d_feature:
    train_transaction[col + '_new'] = train_transaction[col] - train_transaction['Transaction_day']
    test_transaction[col + '_new'] = test_transaction[col] - test_transaction['Transaction_day']


count_list = [[col] + card_feature + ['D1_new'] for col in ['C4', 'C5']]
basis_list = [card_feature + addr_feature + c_feat + ['D1_new'],
              card_feature + addr_feature + ['D1_new']
             ]
day_list =  [[col] + card_feature + ['D1_new'] for col in ['D10_new', 'D15_new']] +\
            [[col] + card_feature + addr_feature for col in ['D2_new', 'D4_new']] +\
            [[col] + card_feature + c_feat for col in ['D4_new', 'D10_new']]
d_new_feature = ["{}_new".format(col) for col in d_feature if col not in ['D1']]


full_list = count_list + basis_list + day_list

feat_rule = c_feat + ['D1_new', 'TransactionID', 'Transaction_day'] + card_feature + addr_feature + d_new_feature
tr_rule = train_transaction[feat_rule + ['isFraud']]
ts_rule = test_transaction[feat_rule]
test_fraud = online_rule_parallel(tr_rule, ts_rule, full_list)
test_fraud.drop_duplicates(['TransactionID'], inplace = True)
# fraud num: 2749
# CPU times: user 3.77 s, sys: 8.86 s, total: 12.6 s
# Wall time: 2min 31s
print("fraud num:", test_fraud.shape[0])
del tr_rule,ts_rule;gc.collect()
from post import online_special_black_parallel
for col in d_feature:
    train_transaction[col + '_new'] = train_transaction[col] - train_transaction['Transaction_day']
    test_transaction[col + '_new'] = test_transaction[col] - test_transaction['Transaction_day']

d_new_feature = ["{}_new".format(col) for col in d_feature if col not in ['D1']]
feat_rule = c_feat + ['D1_new', 'TransactionID', 'Transaction_day'] + card_feature + addr_feature + d_new_feature
tr_rule = train_transaction[feat_rule + ['isFraud']]
ts_rule = test_transaction[feat_rule]


count_list = [[col] + card_feature + ['D1_new'] for col in ['C12']]
day_list =  [[col] + card_feature + ['D1_new'] for col in ['D15_new']] +\
            [[col] + card_feature + addr_feature for col in ['D2_new']]
        
full_list = count_list + day_list

special_fraud = online_special_black_parallel(tr_rule, ts_rule, full_list)
special_fraud.drop_duplicates(['TransactionID'], inplace = True)
print("fraud num:", special_fraud.shape[0])
# fraud num: 3683
# CPU times: user 3.57 s, sys: 1.75 s, total: 5.32 s
# Wall time: 6min 34s
del tr_rule,ts_rule;gc.collect()
from post import online_grey_parallel
for col in d_feature:
    train_transaction[col + '_new'] = train_transaction[col] - train_transaction['Transaction_day']
    test_transaction[col + '_new'] = test_transaction[col] - test_transaction['Transaction_day']

d_new_feature = ["{}_new".format(col) for col in d_feature if col not in ['D1']]
day_list =  [[col] + card_feature + ['D1_new'] for col in ['D11_new']]
#plus D11_new out of research.
full_list = day_list

feat_rule = c_feat + ['D1_new', 'TransactionID', 'Transaction_day'] + card_feature + addr_feature + d_new_feature
tr_rule = train_transaction[feat_rule + ['isFraud']]
ts_rule = test_transaction[feat_rule]
test_grey = online_grey_parallel(tr_rule, ts_rule, full_list)
test_grey.drop_duplicates(['TransactionID'], inplace = True)
print("fraud num:", test_grey.shape[0])
# fraud num: 1279
# CPU times: user 3.95 s, sys: 2.12 s, total: 6.06 s
# Wall time: 6min 59s
del tr_rule,ts_rule;gc.collect()
from post import online_white_parallel
tr_rule = train_transaction[feat_rule + ['isFraud']]
ts_rule = test_transaction[feat_rule]
full_list = [card_feature + addr_feature + c_feat + ['D1_new'], card_feature + c_feat + ['D1_new']]

test_white = online_white_parallel(train_transaction, test_transaction, full_list)
test_white.drop_duplicates(['TransactionID'], inplace = True)
# fraud num: 34369
# CPU times: user 2.47 s, sys: 4.66 s, total: 7.13 s
# Wall time: 2min 41s
print("fraud num:", test_white.shape[0])
del tr_rule,ts_rule;gc.collect()


# begin.
# fraud num: 2749
# begin.
# fraud num: 2039
# begin.
# fraud num: 837
# begin.
# fraud num: 34369
# CPU times: user 10.3 s, sys: 27.2 s, total: 37.5 s
# Wall time: 8min 41s

begin.
fraud num: 3641
begin.
fraud num: 2111
begin.
fraud num: 171
begin.
fraud num: 34369
CPU times: user 5.96 s, sys: 15 s, total: 21 s
Wall time: 6min 44s


In [69]:
%%time
print("begin.")
if 'fraud_guess' in test_transaction.columns:
    test_transaction.drop(['fraud_guess'], axis = 1, inplace = True)
test_transaction['pred'] = oof_test.mean(axis = 1)
test_transaction = pd.merge(test_transaction, test_fraud, on = ['TransactionID'], how = 'left')
test_transaction.fraud_guess.fillna(0, inplace = True)
test_transaction['pred'] = np.where(test_transaction['fraud_guess'] == 1, 1, test_transaction['pred'])

if 'special_fraud_guess' in test_transaction.columns:
    test_transaction.drop(['special_fraud_guess'], axis = 1, inplace = True)
test_transaction = pd.merge(test_transaction, special_fraud, on = ['TransactionID'], how = 'left')
test_transaction.special_fraud_guess.fillna(0, inplace = True)
test_transaction['pred'] = np.where(test_transaction['special_fraud_guess'] == 1, 1, test_transaction['pred'])


if 'grey_guess' in test_transaction.columns:
    test_transaction.drop(['grey_guess'], axis = 1, inplace = True)
test_transaction = pd.merge(test_transaction, test_grey, on = ['TransactionID'], how = 'left')
test_transaction.grey_guess.fillna(0, inplace = True)
test_transaction['pred'] = np.where(test_transaction['grey_guess'] == 1, 1, test_transaction['pred'])

if 'white_guess' in test_transaction.columns:
    test_transaction.drop(['white_guess'], axis = 1, inplace = True)
test_transaction = pd.merge(test_transaction, test_white, on = ['TransactionID'], how = 'left')
test_transaction.white_guess.fillna(0, inplace = True)
test_transaction['pred'] = np.where(test_transaction['white_guess'] == 1, 0, test_transaction['pred'])


from time import localtime, strftime
from sklearn.metrics import roc_auc_score
# score = roc_auc_score(train_transaction['isFraud'],oof_train)
score = roc_auc_score(train_transaction['isFraud'],oof_train)
mean_score = np.mean(score_list[0:5])
sub = pd.DataFrame([])
strategy = 'KFold'
sub['TransactionID'] = test_transaction['TransactionID']
sub['isFraud'] = test_transaction['pred']
black_num = "black:{}".format(test_transaction.loc[(test_transaction['fraud_guess'] == 1)].shape[0])
special_num = "special:{}".format(test_transaction.loc[(test_transaction['special_fraud_guess'] == 1)].shape[0])
white_num = "white:{}".format(test_transaction.loc[(test_transaction['white_guess'] == 1)].shape[0])
grey_num = "grey:{}".format(test_transaction.loc[(test_transaction['grey_guess'] == 1)].shape[0])
sub.to_csv('/root/ieee/sub/{}_{}_oof_{}_mean_{}_post_{}_{}_{}_{}.csv'.format(strategy,
                                                    strftime("%Y_%m%d_%H%M", localtime()), 
                                                    np.around(score,5), np.around(mean_score, 5),
                                                    black_num, special_num, white_num, grey_num), index = None)



begin.
CPU times: user 47.8 s, sys: 1min 28s, total: 2min 16s
Wall time: 2min 16s


In [ ]:
import numpy as np
import pandas as pd
import gc
import time
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

def null_importance(train_df, train_features, nb_runs = 20):
    def get_feature_importances(data, train_features, shuffle = False):
        y = data['isFraud'].copy()
        if shuffle:
        # Here you could as well use a binomial distribution
            y = data['isFraud'].copy().sample(frac=1.0)
        dtrain = lgb.Dataset(data[train_features], y, free_raw_data=False, silent=False)
        params = {'num_leaves': 450,
          'min_child_weight': 0.03,
          'feature_fraction': 0.37,
          'bagging_fraction': 0.42,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.01,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.39,
          'reg_lambda': 0.65,
          'random_state': 47,
          # 'seed':0
         }
        clf = lgb.train(params=params,train_set=dtrain,num_boost_round=100)
        preds = clf.predict(data[train_features])
        # Get feature importances
        imp_df = pd.DataFrame()
        imp_df["feature"] = list(train_features)
        imp_df["importance_gain"] = clf.feature_importance(importance_type='gain')
        imp_df["importance_split"] = clf.feature_importance(importance_type='split')
        imp_df['trn_score'] = roc_auc_score(y, preds)
        del dtrain;gc.collect()
        return imp_df
    np.random.seed(817)
    # Get the actual importance, i.e. without shuffling
    actual_imp_df = get_feature_importances(data = train_df, train_features = train_features, shuffle=False)
    null_imp_df = pd.DataFrame()
    start = time.time()
    dsp = ''
    for i in range(nb_runs):
        print('round:', i)
    # Get current run importances
        imp_df = get_feature_importances(data=train_df, train_features = train_features, shuffle=True)
        imp_df['run'] = i + 1
    # Concat the latest importances with the old ones
        null_imp_df = pd.concat([null_imp_df, imp_df], axis=0)
    # Erase previous message
        for l in range(len(dsp)):
            print('\b', end='', flush=True)
    # Display current run and time used
    spent = (time.time() - start) / 60
    dsp = 'Done with %4d of %4d (Spent %5.1f min)' % (i + 1, nb_runs, spent)
    print(dsp, end='', flush=True)
    feature_scores = []
    for _f in actual_imp_df['feature'].unique():
        f_null_imps_gain = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_gain'].values
        f_act_imps_gain = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_gain'].mean()

        gain_score = np.log(1e-10 + f_act_imps_gain / (1 + np.percentile(f_null_imps_gain, 75)))  # Avoid didvide by zero
        f_null_imps_split = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_split'].values
        f_act_imps_split = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_split'].mean()
        split_score = np.log(1e-10 + f_act_imps_split / (1 + np.percentile(f_null_imps_split, 75)))  # Avoid didvide by zero
        feature_scores.append((_f, split_score, gain_score))

    scores_df = pd.DataFrame(feature_scores, columns=['feature', 'split_score', 'gain_score'])
    new_list = scores_df.sort_values(by=['gain_score'],ascending=False).reset_index(drop=True)
    return new_list




In [68]:
from model import kfold_lightgbm
var_result = null_importance(train_transaction, used, 50)
null_feat = list(var_result.loc[var_result['gain_score'] >= 0, 'feature'])
print(len(null_feat))
#669?
oof_train, oof_test, score_list = kfold_lightgbm(train_transaction, test_transaction, null_feat)

round: 0
round: 1
round: 2
round: 3
round: 4
round: 5
round: 6
round: 7
round: 8
round: 9
round: 10
round: 11
round: 12
round: 13
round: 14
round: 15
round: 16
round: 17
round: 18
round: 19
round: 20
round: 21
round: 22
round: 23
round: 24
round: 25
round: 26
round: 27
round: 28
round: 29
round: 30
round: 31
round: 32
round: 33
round: 34
round: 35
round: 36
round: 37
round: 38
round: 39
round: 40
round: 41
round: 42
round: 43
round: 44
round: 45
round: 46
round: 47
round: 48
round: 49
Done with   50 of   50 (Spent  53.6 min)511
train.shape = (587087, 774), test.shape = (506691, 778)
############################################################ fold = 1 / 5
####### cur time = 2019/09/22 21:16:25
Training until validation scores don't improve for 100 rounds.
[500]	valid_0's auc: 0.933044
[1000]	valid_0's auc: 0.940294
Early stopping, best iteration is:
[1267]	valid_0's auc: 0.94077
period: [     0      1      2 ... 117415 117416 117417] , the score is 0.940769735220405
###################